<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment_Feedforward_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
import shutil
import tensorflow as tf
import string
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
import pandas as pd
import glob
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics.pairwise import cosine_similarity

print(tf.__version__)


2.6.0


# Part 1: IMDB sentiment analysis

## Loading the dataset

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
data = tf.keras.utils.get_file ('aclImdb_v1',
                                url,
                                untar = True,
                                cache_dir = '.',
                                cache_subdir = '')
data_dir = os.path.join (os.path.dirname(data), 'aclImdb')
print(os.listdir(data_dir))

['imdb.vocab', 'test', 'README', 'imdbEr.txt', 'train']


In [ ]:
train_dir = os.path.join (data_dir, 'train')
test_dir = os.path.join (data_dir, 'test')
print(os.listdir(train_dir))

['neg', 'urls_neg.txt', 'urls_pos.txt', 'unsup', 'pos', 'urls_unsup.txt', 'unsupBow.feat', 'labeledBow.feat']


In [ ]:
# We only use files in the two folders: pos, and neg, so let's remove other files
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
# Loading data from the directory
batch_size = 32
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('aclImdb/test',
                                                       batch_size = batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


## Text representation

In [ ]:
def custom_preprocessing (text):
  lowercase = tf.strings.lower (text)
  stripped_html = tf.strings.regex_replace (lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), 
                                  '')
  
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(standardize = custom_preprocessing,
                                           max_tokens = max_features,
                                           output_mode = 'int',
                                           output_sequence_length = sequence_length)
# Extracting features for vectorizing using training set
train_text = raw_train.map (lambda x, y: x)
vectorize_layer.adapt(train_text)

# Defining a function for fitting vectorizer function/layer to vectorize text (review)
def fitting_vectorizer (text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer (text), label

# storing text batch and label batch
text_batch, label_batch = next(iter(raw_train))

## print an instance with vectorized review and label for observing
print ('REVIEW:', text_batch[0])
print('LABEL:', raw_train.class_names[label_batch[0]] )


REVIEW: tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />

In [ ]:
# print an example of vectorized data
print ('Vocabulary size: ', len(vectorize_layer.get_vocabulary()))
for i in range (90, 100):
  print ('{} ------> {}'.format(i, vectorize_layer.get_vocabulary()[i]))

Vocabulary size:  10000
90 ------> made
91 ------> movies
92 ------> then
93 ------> them
94 ------> films
95 ------> way
96 ------> make
97 ------> any
98 ------> could
99 ------> too


In [ ]:
train = raw_train.map(fitting_vectorizer)
val = raw_val.map(fitting_vectorizer)
test = raw_test.map(fitting_vectorizer)


In [ ]:
# Configure the dataset for performance
autotune = tf.data.AUTOTUNE
train = train.cache().prefetch (buffer_size = autotune)
val = val.cache().prefetch (buffer_size = autotune)
test = test.cache().prefetch (buffer_size = autotune)

## Building a neural network classifier

In [ ]:
# Defining an evaluation metric function
def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred, average='binary')
  recall = sklearn.metrics.recall_score(y_true, y_pred, average='binary')
  f1 = sklearn.metrics.f1_score(y_true, y_pred , average='binary')
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

### With different numbers of embedding dimentions

In [ ]:
# Creating the model
embedding_dim = [16, 28, 50]
for n in embedding_dim:
  print ("========= embedding vectors'size= %s ============" %n)
  model = tf.keras.Sequential([layers.Embedding(max_features + 1, n, name="embedding"),
                              layers.Dropout(0.2),
                              layers.GlobalAveragePooling1D(),
                              layers.Dropout(0.2),
                              layers.Dense(1, activation = 'sigmoid')])
  print(model.summary())

  # configure the model uisng optimizer and loss function
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = 'adam',
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.5 )) 
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  # pred_label = tf.argmax(model.predict(test),1)
  pred_label = (model.predict(test) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test], axis=0)

  loss, accuracy = model.evaluate(test)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))
  printing_eval_scores (true_label, pred_label, report=True)

========= embedding vectors'size= 16 ============
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


625/625 [==============================] - 10s 15ms/step - loss: 0.6636 - binary_accuracy: 0.6902 - val_loss: 0.6132 - val_binary_accuracy: 0.7704
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5475 - binary_accuracy: 0.8018 - val_loss: 0.4969 - val_binary_accuracy: 0.8216
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.4437 - binary_accuracy: 0.8448 - val_loss: 0.4190 - val_binary_accuracy: 0.8486
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3775 - binary_accuracy: 0.8651 - val_loss: 0.3728 - val_binary_accuracy: 0.8612
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3351 - binary_accuracy: 0.8802 - val_loss: 0.3444 - val_binary_accuracy: 0.8674
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3051 - binary_accuracy: 0.8884 - val_loss: 0.3254 - val_binary_accuracy: 0.8716
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss:

From the performance of models with different embedding size (the number of dimmensions), we can see their performances are not much different. Among them, the model with embedding vectors'size= 28 has the highest performance (F1 score: 0.874), slightly higher than the models with embedding vectors'size of 16 or 50.

### With different dropout 

In [ ]:
# Creating the model
embedding_dim = 16
dropouts = [0.0, 0.1, 0.2, 0.3]
for i in dropouts:
  print ("========= dropout = %s ============" %i)
  model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim,  name="embedding"),
                              layers.Dropout(i),
                              layers.GlobalAveragePooling1D(),
                              layers.Dropout(0.2),
                              layers.Dense(1,activation = 'sigmoid')])
  print(model.summary())

  # configure the model uisng optimizer and loss function
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = 'adam',
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.5 )) 
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  pred_label = (model.predict(test) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test], axis=0)

  loss, accuracy = model.evaluate(test)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))
  printing_eval_scores (true_label, pred_label, report=True)

========= dropout = 0.0 ============
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


625/625 [==============================] - 3s 4ms/step - loss: 0.6624 - binary_accuracy: 0.6939 - val_loss: 0.6104 - val_binary_accuracy: 0.7748
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.5418 - binary_accuracy: 0.8044 - val_loss: 0.4902 - val_binary_accuracy: 0.8242
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.4356 - binary_accuracy: 0.8496 - val_loss: 0.4121 - val_binary_accuracy: 0.8500
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3699 - binary_accuracy: 0.8697 - val_loss: 0.3669 - val_binary_accuracy: 0.8610
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3272 - binary_accuracy: 0.8818 - val_loss: 0.3395 - val_binary_accuracy: 0.8696
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2963 - binary_accuracy: 0.8913 - val_loss: 0.3211 - val_binary_accuracy: 0.8724
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0

Setting a dropout layer is for preventing overfitting, and save computational resource. We experiment different dropout sizes: no dropout, 10% dropout, 20%, and 30% dropout. Generally, the models with different dropout rates have quite similar performance. Dropout rate of 0.3 causes the performance to drop a bit (F1 score: 0.873) while no dropout, dropout rate of 0.1 and 0.2 do not affect the model performance regarding F1 score. 

### Adding a Dense layer

In [ ]:
# Creating the model
embedding_dim = 16
dropout =  0.1
activations = ['relu', 'softmax', 'sigmoid']
for f in activations:
  print ("========= activation function = %s ============" %f)
  model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim,  name="embedding"),
                              layers.Dropout(dropout),
                              layers.GlobalAveragePooling1D(),
                              layers.Dropout(dropout),
                              layers.Dense(32, activation= f),
                              layers.Dense(1,activation = 'sigmoid')])
  print(model.summary())

  # configure the model uisng optimizer and loss function
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = 'adam',
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.5 )) 
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  pred_label = (model.predict(test) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test], axis=0)

  loss, accuracy = model.evaluate(test)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))
  printing_eval_scores (true_label, pred_label, report=True)

========= activation function = relu ============
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout_14 (Dropout)         (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_7 ( (None, 16)                0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 160,593
Trainable params: 160,593
Non-trainable params: 

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


625/625 [==============================] - 4s 6ms/step - loss: 0.5472 - binary_accuracy: 0.7424 - val_loss: 0.3561 - val_binary_accuracy: 0.8552
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2924 - binary_accuracy: 0.8837 - val_loss: 0.3020 - val_binary_accuracy: 0.8738
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2256 - binary_accuracy: 0.9131 - val_loss: 0.2988 - val_binary_accuracy: 0.8764
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.1854 - binary_accuracy: 0.9327 - val_loss: 0.3118 - val_binary_accuracy: 0.8738
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 0.1549 - binary_accuracy: 0.9448 - val_loss: 0.3342 - val_binary_accuracy: 0.8694
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.1324 - binary_accuracy: 0.9553 - val_loss: 0.3559 - val_binary_accuracy: 0.8680
Epoch 7/10
625/625 [==============================] - 4s 7ms/step - loss: 0

We add one more Dense layer before the last Dense layer, and experiment different nonlinear functions: ReLU, softmax, and sigmoid. The model with ReLU activation performs worst with F1 score = 0.836 on the test set. The model with softmax activation performs the best with a F1 score of 0.871, followed by the model with sigmoid activation (F1 score = 0.864).

### With different Batch size

In [ ]:
# Loading data from the directory
batch_size = 64
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('aclImdb/test',
                                                       batch_size = batch_size)

# storing text batch and label batch
text_batch, label_batch = next(iter(raw_train))

## print an instance with vectorized review and label for observing
print ('REVIEW:', text_batch[0])
print('LABEL:', raw_train.class_names[label_batch[0]] )


# Creating the model
embedding_dim = 16
dropout =  0.1
activation =  'softmax'

print ("======== activation function = {}, dropout = {}, batch size = {} ============".format(activation, dropout, batch_size ))
model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim, name="embedding"),
                            layers.Dropout(dropout),
                            layers.GlobalAveragePooling1D(),
                            layers.Dropout(dropout),
                            layers.Dense(32, activation= activation),
                            layers.Dense(1)])
print(model.summary())

# configure the model uisng optimizer and loss function
model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
              optimizer = 'adam',
              metrics = tf.metrics.BinaryAccuracy(threshold = 0.5 )) 
# training the model
epochs = 10
history = model.fit(train,
                    validation_data = val,
                    epochs = epochs)
# testing the model
pred_label = (model.predict(test) > 0.5).astype("int32")
true_label = np.concatenate([y for x, y in test], axis=0)

loss, accuracy = model.evaluate(test)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))
printing_eval_scores (true_label, pred_label, report=True)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
REVIEW: tf.Tensor(b"First of all, I liked very much the central idea of locating the '' intruders'', Others in the fragile Self, on various levels - mainly subconscious but sometimes more allegorical. In fact the intruders are omnipresent throughout the film : in the Swiss-French border where the pretagonist leads secluded life; in the his recurring daydream and nightmare; inside his ailing body after heart transplantation.... In the last half of the film, he becomes intruder himself, returning in ancient french colony in the hope of atoning for the past. <br /><br />The overall tone is bitter rather than pathetic, full of regrets and guilts, sense of failure being more or less dominant. This is a quite grim picture of an old age, ostensibly self-dependent but hopelessly void and lonely inside. The

(0.86992, 0.8967736399519478, 0.83608, 0.8653639148795231)

Previously, we have built the model Sequence_8 with a batch size of 32, dropout = 0.1, activation function = softmax, and it achieves F1 score = 0.871. In the above model (Sequence_9), we use the same architecture, but train it on a batch size of 64 to investigate whether batch size impacts performance of the model. We found that F1 score drops a bit to 0.865, meaning that batch size of 32 or 64 does not significantly impact the model's performance.

### With different training algorithms
Here we mostly focus on adapting learning rate methods to train the models

In [ ]:
# configure the model uisng optimizer and loss function
optimizers = ['adagrad', 'rmsprop', 'adam']

print ("======== activation function = {}, dropout = {}, batch size = {} ============".format(activation, dropout, batch_size ))
for opt in optimizers:
  print( '========== optimizer = %s' %opt)
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = opt,
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.5 )) 
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  pred_label = (model.predict(test) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test], axis=0)

  loss, accuracy = model.evaluate(test)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))
  printing_eval_scores (true_label, pred_label, report=True)



======== activation function = softmax, dropout = 0.1, batch size = 64 ============
========== optimizer = adagrad
Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 0.1585 - binary_accuracy: 0.9485 - val_loss: 0.3038 - val_binary_accuracy: 0.8772
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1577 - binary_accuracy: 0.9495 - val_loss: 0.3040 - val_binary_accuracy: 0.8770
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1582 - binary_accuracy: 0.9485 - val_loss: 0.3041 - val_binary_accuracy: 0.8770
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1581 - binary_accuracy: 0.9502 - val_loss: 0.3041 - val_binary_accuracy: 0.8766
Epoch 5/10
625/625 [==============================] - 3s 6ms/step - loss: 0.1582 - binary_accuracy: 0.9495 - val_loss: 0.3043 - val_binary_accuracy: 0.8768
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1578 - binary_accuracy: 0.949

We explore three different optimizers: Adam, adagrad, and rmsprop for training the model with activation function = softmax, dropout = 0.1, batch size = 64. We found that adagrad opimizer helps the model achieve higher performance (F1 score: 0.865) than the model trained with Adam (F1 score = 0.856), and rmsprop (F1 score = 0.850).

## Word Embedding

In [ ]:
# Getting the weights that we have trained on our model, and their corresponding vocabulary 
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
# Defining a function to find the most similar words to a certain chosen word.
def Find_similar_w (word, n):
  all_cos_sim = {}
  idx = vocab.index(word)
  weight = weights[idx]
  for i in range(len(weights)-1):
    cosine_sim = cosine_similarity(weight.reshape(1, -1), weights[i].reshape(1, -1))
    all_cos_sim[vocab[i]] = cosine_sim
  # Sorting the dictionary in descending order
  sorted_cos = {k:v for k, v in sorted(all_cos_sim.items(), key = lambda item: item[1], reverse=True)}
  print ("Top {} most similar with '{}' \n".format(n, word))
  for k, v in list(sorted_cos.items())[:n]:
    print ('{} =====> similarity score: {}'. format(k,v))
  return  sorted_cos

most_similar_w = Find_similar_w (word = 'boring', n = 20)


Top 20 most similar with 'boring' 

boring =====> [[1.]]
disappointment =====> [[0.99934846]]
unconvincing =====> [[0.99932355]]
meyer =====> [[0.99920887]]
dull =====> [[0.9991085]]
410 =====> [[0.9991035]]
worst =====> [[0.9990882]]
terrible =====> [[0.9990096]]
badness =====> [[0.99898285]]
cena =====> [[0.9989134]]
pointless =====> [[0.99889576]]
nothing =====> [[0.9988869]]
downhill =====> [[0.9987695]]
lacks =====> [[0.998716]]
giant =====> [[0.9987079]]
unwatchable =====> [[0.99868995]]
worse =====> [[0.998679]]
thunderbirds =====> [[0.9986668]]
options =====> [[0.9986569]]
seconds =====> [[0.9986452]]


We make use of the weights that have been trained in our network to find the most similar words to the chosen words. First, we obtain and store these weights into a list. Meanwhile, we store the vocabulary which corresponds to weights with index into another list. Next, we select a word, 'boring', and get its trained weight using index. Then, by looping over weights in the weight list, we compute cosine similarity between the weight of the word 'boring', and other words' weights. The higher cosine similarity scores are, the more similar the word pairs are. The above result shows the top 10 most similar words with the word 'boring'. Most of them make a lot of sense such as 'disappointment', 'unconvincing', 'dull', 'worst', 'terrible'. The term 'meyer', and '410' seem not related, but they are likely used with the term 'boring'.

## Comparing with a Logistic Regression model

In [ ]:
# loading the ds
def Getdata(dir):
  review, label = [], []
  for folder in glob.glob (dir+'/*'):
    for file in glob.glob(folder+'/*'): 
      fo = open(file)
      doc = fo.read()
      review.append (doc)
      if 'pos' in file:
        label.append(1)
      elif 'neg' in file:
        label.append(0)
  df = pd.DataFrame(zip(review,label), columns = ['review', 'label'])
  return df
  
train_df = Getdata(train_dir)
test_df = Getdata(test_dir)
# Spliting the dataset for training and testing
X_train, X_val, y_train, y_val = train_test_split (train_df['review'],train_df['label'], train_size = 0.8, random_state = 42, shuffle = True)
X_test, y_test = test_df['review'], test_df['label']
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_val, y_val: ', X_val.shape, y_val.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape) 

Shapes of X_train, y_train:  (20000,) (20000,)
Shapes of X_val, y_val:  (5000,) (5000,)
Shapes of X_test, y_test:  (25000,) (25000,)


In [ ]:

def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred)
  recall = sklearn.metrics.recall_score(y_true, y_pred)
  f1 = sklearn.metrics.f1_score(y_true, y_pred)
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

### With Countvectorizer text presentation

In [ ]:

# Vectorizing the documents
vectorizer = CountVectorizer(binary = True)
X_train_count = vectorizer.fit_transform(X_train.to_list())
X_val_count = vectorizer.transform(X_val.to_list())
X_test_count = vectorizer.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train_count.shape, y_train.shape)
print ('Shapes of X_val, y_val: ', X_val_count.shape, y_val.shape)
print ('Shapes of X_test, y_test: ', X_test_count.shape, y_test.shape)

# Sklearn Logistic Regression Model
sk_lr_1 = LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train_count, y_train )
y_predict = sk_lr_1.predict(X_test_count)

# Model performing
## on training set
print('Model performance with Countvectorizer: \non validation set:')
printing_eval_scores (y_val, sk_lr_1.predict(X_val_count))

## on test set
print('\n===========================')
print('on test set:')
printing_eval_scores (y_test, y_predict, report = True)

Shapes of X_train, y_train:  (20000, 68468) (20000,)
Shapes of X_val, y_val:  (5000, 68468) (5000,)
Shapes of X_test, y_test:  (25000, 68468) (25000,)
Model performance with Countvectorizer: 
on validation set:
accuracy score: 0.872
precision score: 0.862
recall score: 0.883
F1 score: 0.873

on test set:
accuracy score: 0.866
precision score: 0.866
recall score: 0.866
F1 score: 0.866
              precision    recall  f1-score   support

           0       0.87      0.87      0.87     12500
           1       0.87      0.87      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



(0.8658, 0.8659463785514205, 0.8656, 0.8657731546309262)

### With tf-idf text presentation

In [ ]:
# Vectorizing the documents
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train.to_list())
X_val_tfidf = tfidf.transform(X_val.to_list())
X_test_tfidf = tfidf.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train_tfidf.shape, y_train.shape)
print ('Shapes of X_val, y_val: ', X_val_tfidf.shape, y_val.shape)
print ('Shapes of X_test, y_test: ', X_test_tfidf.shape, y_test.shape)

# Sklearn Logistic Regression Model
sk_lr_2 = LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train_tfidf, y_train )
y_predict = sk_lr_2.predict(X_test_tfidf)

# Model performing
## on training set
print('Model performance with tfidf: \non validation set:')
printing_eval_scores (y_val, sk_lr_2.predict(X_val_tfidf))

## on test set
print('\n===========================')
print('on test set:')
printing_eval_scores (y_test, y_predict, report = True)

Shapes of X_train, y_train:  (20000, 68468) (20000,)
Shapes of X_val, y_val:  (5000, 68468) (5000,)
Shapes of X_test, y_test:  (25000, 68468) (25000,)
Model performance with tfidf: 
on validation set:
accuracy score: 0.893
precision score: 0.888
recall score: 0.898
F1 score: 0.893

on test set:
accuracy score: 0.882
precision score: 0.881
recall score: 0.883
F1 score: 0.882
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



(0.882, 0.8811462324393359, 0.88312, 0.8821320121463961)

By comparing the two Logistic Regression models with two different text representations: Countvectorizer, and tfidf. We found that the model using tfidf to represent the reviews achieved higher performance with F1 score  = 0.882, higher than any models we built on tensorflow, while the another with countvectorizer has a bit lower performance (F1 = 0.866). In the future, we may need to try different text representations for the models on tensorflow.

# Part 2: Multiclass classification - Stackoverflow DS

## Loading the dataset

In [ ]:
url_2 = 'http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
train_dir = tf.keras.utils.get_file ('train',
                                url_2,
                                untar = True,
                                cache_dir = '.',
                                cache_subdir = '')
test_dir = tf.keras.utils.get_file ('test',
                                url_2,
                                untar = True,
                                cache_dir = '.',
                                cache_subdir = '')

print(os.listdir(train_dir))
print(os.listdir(test_dir))


['java', 'javascript', 'python', 'csharp']
['java', 'javascript', 'python', 'csharp']


In [ ]:
# Loading data from the directory
batch_size = 32
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('test',
                                                       batch_size = batch_size)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


## Text representation

In [ ]:
def custom_preprocessing (text):
  lowercase = tf.strings.lower (text)
  stripped_html = tf.strings.regex_replace (lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), 
                                  '')
  
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(standardize = custom_preprocessing,
                                           max_tokens = max_features,
                                           output_mode = 'int',
                                           output_sequence_length = sequence_length)
# Extracting features for vectorizing using training set
train_text = raw_train.map (lambda x, y: x)
vectorize_layer.adapt(train_text)

# Defining a function for fitting vectorizer function/layer to vectorize text (review)
def fitting_vectorizer (text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer (text), label

# storing text batch and label batch
text_batch, label_batch = next(iter(raw_train))

## print an instance with vectorized review and label for observing
print ('text:', text_batch[0])
print('label:', raw_train.class_names[label_batch[0]] )

text: tf.Tensor(b'"unit testing of setters and getters teacher wanted us to do a comprehensive unit test. for me, this will be the first time that i use junit. i am confused about testing set and get methods. do you think should i test them? if the answer is yes; is this code enough for testing?..  public void testsetandget(){.    int a = 10;.    class firstclass = new class();.    firstclass.setvalue(10);.    int value = firstclass.getvalue();.    assert.asserttrue(""error"", value==a);.  }...in my code, i think if there is an error, we can\'t know that the error is deriving because of setter or getter."\n', shape=(), dtype=string)
label: java


In [ ]:
train = raw_train.map(fitting_vectorizer)
val = raw_val.map(fitting_vectorizer)
test = raw_test.map(fitting_vectorizer)

In [ ]:
# Configure the dataset for performance
autotune = tf.data.AUTOTUNE
train = train.cache().prefetch (buffer_size = autotune)
val = val.cache().prefetch (buffer_size = autotune)
test = test.cache().prefetch (buffer_size = autotune)

## Building a neural network multiclass classifier

In [ ]:
# Defining an evaluation metric function
def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred, average='macro')
  recall = sklearn.metrics.recall_score(y_true, y_pred, average='macro')
  f1 = sklearn.metrics.f1_score(y_true, y_pred , average='macro')
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

In [ ]:
# Creating the model
embedding_dim = 16
dropout =  0.1
activation = 'relu'

print ("======== activation function = {}, dropout = {}, batch size = {} ============".format(activation, dropout, batch_size ))
model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim, name="embedding_2"),
                            layers.Dropout(dropout),
                            layers.GlobalAveragePooling1D(),
                            layers.Dropout(dropout),
                            layers.Dense(32, activation= activation),
                            layers.Dense(4)])
print(model.summary())

# configure the model uisng optimizer and loss function
model.compile(loss = losses.SparseCategoricalCrossentropy(from_logits = True),
              optimizer = 'adam',
              metrics = 'accuracy') 
# training the model
epochs = 10
history = model.fit(train,
                    validation_data = val,
                    epochs = epochs)
# testing the model
pred_label = tf.argmax(model.predict(test),1)
true_label = np.concatenate([y for x, y in test], axis=0)

loss, accuracy = model.evaluate(test)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))
printing_eval_scores (true_label, pred_label, report=True)

======== activation function = relu, dropout = 0.1, batch size = 32 ============
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_22 (Dropout)         (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_11  (None, 16)                0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 132       
Total params: 160,692
Trainable params

(0.77275, 0.7734619159160259, 0.77275, 0.7710657825470874)